# get_freq_mutations test colon

In [1]:
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt

import CPTAC.Colon as co

Welcome to the CPTAC data service package. Available datasets may be
viewed using CPTAC.list_data(). In order to access a specific data
set, import a CPTAC subfolder using either 'import CPTAC.Dataset' or
'from CPTAC import Dataset'.
******
Version: 0.3.1
******
Loading Colon CPTAC data:
Loading clinical data...
Loading miRNA data...
Loading mutation data...
Loading mutation_binary data...
Loading phosphoproteomics_normal data...
Loading phosphoproteomics_tumor data...
Loading proteomics_normal data...
Loading proteomics_tumor data...
Loading transcriptomics data...


In [7]:
def get_frequently_mutated(somatic_df, omics_mutations_df, cutoff=.1, show_percentage=False):  
    import operator
    unique_genes = somatic_df['Gene'].unique() # get series of all mutated genes
    freq_mutated = []
    gene_and_freq_d = {}
    percentage = 0.0
    
    # get total tumor patients
    tumors = omics_mutations_df.loc[omics_mutations_df['Sample_Status'] == 'Tumor']
    total_tumor_patients = len(tumors)
    

    #find sample (col or index) in gene_mutated (samples represent patients)
    #gene_mutated: Endometrial and Colon samples found in column 1, Ovarian found in index
    gene = 'PTEN'
    gene_mutated = somatic_df.loc[somatic_df['Gene'] == gene]
    ovarian = False
    sample = 0
    if gene_mutated.columns[0] == 'Gene':
        sample = gene_mutated.index
        ovarian = True
        
    #find frequently mutated
    for gene in unique_genes:
        gene_mutated = somatic.loc[somatic['Gene'] == gene]
        num_gene_mutated = len(gene_mutated[gene_mutated.columns[sample]].unique())     
        percentage = (num_gene_mutated / total_tumor_patients)
        if percentage > cutoff:
            gene_and_freq_d[gene] = percentage

    sorted_d = sorted(gene_and_freq_d.items(), key=operator.itemgetter(1), reverse=True)  
    
    
    for i in range(0,len(sorted_d)):
        certain_tuple = sorted_d[i]
        gene, percent_mutated = certain_tuple
        if show_percentage == True:
            string_gene_percent = gene + ': %' + str('%.2f'%percent_mutated)
            freq_mutated.append(string_gene_percent)
        else:
            freq_mutated.append(gene)
  
    return freq_mutated

# find number of tumor patients

In [16]:
somatic = co.get_mutations()
prot = co.get_proteomics()
clinical = co.get_clinical()
total = clinical.loc[clinical['Tumor.Status'] == 'With tumor']
print(len(total)) #NOT RIGHT TOTAL 16
gene_mutated = somatic.loc[somatic['Gene'] == 'PTEN']
gene_mutated.head()

16


,SampleID,Gene,Mutation,Location
88765,05CO050,PTEN,frameshift deletion,N496Mfs*20
88764,05CO050,PTEN,frameshift deletion,N323Mfs*20
88763,05CO050,PTEN,frameshift deletion,N126Mfs*20
88754,11CO007,PTEN,nonsynonymous SNV,D197N
88755,11CO007,PTEN,nonsynonymous SNV,D24N


In [4]:
# test to finding sample
gene = 'PTEN'
gene_mutated = somatic.loc[somatic['Gene'] == gene]
sample = 0
if gene_mutated.columns[0] == 'Gene':
    sample = gene_mutated.index
sample

0

In [25]:
#find all tumor patients
gene = 'TP53'
omics_mutations = co.append_mutations_to_omics(mutation_genes=gene, omics_df=prot, omics_genes=gene)
tumors = omics_mutations.loc[omics_mutations['Sample_Status'] == 'Tumor']
total_tumor_count = len(tumors)
print(total_tumor_count) #97
omics_mutations.head() 

97


,TP53_proteomics,TP53_Mutation,TP53_Location,Sample_Status
index,,,,
01CO005,-0.489,frameshift insertion,E287Wfs*10,Tumor
01CO006,-0.796,Wildtype_Tumor,No_mutation,Tumor
01CO008,NaN,Wildtype_Tumor,No_mutation,Tumor
01CO013,-1.390,nonsynonymous SNV,C143Y,Tumor
01CO014,NaN,nonsynonymous SNV,R273C,Tumor


Test 1 colon: 

In [10]:
list1 = get_frequently_mutated(somatic, omics_mutations, cutoff=.25, show_percentage=True)

In [12]:
list1

['APC: %0.85',
 'TTN: %0.62',
 'TP53: %0.58',
 'MUC16: %0.40',
 'KRAS: %0.36',
 'SYNE1: %0.30',
 'OBSCN: %0.30',
 'CCDC168: %0.30',
 'FAT3: %0.29',
 'MUC5B: %0.28',
 'CSMD3: %0.27',
 'FAT4: %0.27',
 'DNAH5: %0.27',
 'ZNF469: %0.26',
 'RYR1: %0.26',
 'ACVR2A: %0.26']

In [26]:
#How many patients in colon df?
tumors = omics_mutations.loc[omics_mutations['Sample_Status'] == 'Tumor']
total_tumor_patients = len(tumors)
print('total_tumor_patients', total_tumor_patients)

print('freq_mutated: ',len(list1))
total_genes = somatic['Gene'].unique()
print('total_genes: ',len(total_genes))

total_tumor_patients 97
freq_mutated:  16
total_genes:  15175


In [52]:
# check genes in freq_mutated list
gene = 'APC'
total_tumor_patients = 97
cutoff = .1
gene_mutated = somatic.loc[somatic['Gene'] == gene]
num_gene_mutated = len(gene_mutated)
print(num_gene_mutated, ' / ', total_tumor_patients, ' ', (num_gene_mutated / total_tumor_patients))
if (num_gene_mutated / total_tumor_patients) > cutoff:
    print('true')
else:
    print('false')
gene_mutated.head()

387  /  97   3.9896907216494846
true


,SampleID,Gene,Mutation,Location
7325,01CO001,APC,stopgain,Q1310X
7327,01CO001,APC,stopgain,Q1328X
7326,01CO001,APC,stopgain,Q1328X
7450,01CO005,APC,frameshift deletion,S1407Vfs*7
7451,01CO005,APC,frameshift deletion,S1407Vfs*7


In [85]:
#check to see if mulitple mutations in one gene
gene = 'APC'
gene_mutated = somatic.loc[somatic['Gene'] == gene]#series of mutations
gene_mutated.rename(columns={ gene_mutated.iloc[0]: "sample" }, inplace = True)

patients = gene_mutated.loc[gene_mutated['sample']]#series of ids
#print('num of patients with mutation counted once: ',len(patients.unique()))


#check each patient counted once #fixme check omics_mutations? 
#print(gene_mutated['SampleID'].value_counts()) #gives num rows for each patient, descending order

#gene_mutated.columns[0]
#new = patients[patients == 'C3L-00586'] #gets index for certain patient
#print(new)
gene_mutated

TypeError: 'Series' objects are mutable, thus they cannot be hashed

In [15]:
#test gene_mutated.columns[sample]].unique())
gene_and_freq_d = {}
unique_genes = ['PTEN']
#somatic['Gene'].unique()

for gene in unique_genes:
    gene_mutated = somatic.loc[somatic['Gene'] == gene] 
    count_gene_once = gene_mutated.iloc[:,[0]]
    num_gene_mutated = gene_mutated.loc[gene_mutated.columns[sample]]     
    
    """
    percentage = (num_gene_mutated / total_tumor_patients)
    if percentage > cutoff:
        gene_and_freq_d[gene] = percentage
        """

AttributeError: 'str' object has no attribute 'unique'

test2: using list1, test if all returned

In [46]:
list1_somatic = somatic.loc[somatic['Gene'] == list1]

ValueError: Lengths must match to compare